In [1]:
# comment out this line Massimo it is for me only
!pip install scikit-image
!pip install PIL

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
from IPython.display import display 
from PIL import Image

D:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# load imgs
X = []
for filename in os.listdir('train/'):
    X.append(img_to_array(load_img('train/'+filename)))
X = np.array(X, dtype = float)
X_train = 1.0 / 255 * X

inception = keras.applications.nasnet.NASNetLarge(weights='imagenet', include_top=True)
inception.graph = tf.get_default_graph()

In [5]:
#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
embed_input = Input(shape=(1000,))
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [6]:
def inception_embeddings(grays):
    grayresized = []
    for i in grays:
#         NAS uses 331*331
        i = resize(i, (331, 331, 3), mode='constant')
        grayresized.append(i)
    grayresized = np.array(grayresized)
    grayresized = preprocess_input(grayresized)
    with inception.graph.as_default():
        embed = inception.predict(grayresized)
    return embed

# augmentation
aug = ImageDataGenerator(
        shear_range=0.5,
        zoom_range=0.5,
        rotation_range=10,
        horizontal_flip=True)


def image_gen(batch_size):
    for batch in aug.flow(X_train, batch_size=batch_size):
        grays = gray2rgb(rgb2gray(batch))
        embed = inception_embeddings(grays)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, inception_embeddings(grays)], Y_batch)


#Train model
batch_size = 15
model.compile(optimizer='adam', loss='mse')
model.fit_generator(image_gen(batch_size), epochs=5, steps_per_epoch=10)

Epoch 1/5


D:\Anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


10/10 [==============================] - 698s 70s/step - loss: 0.4219
Epoch 2/5
10/10 [==============================] - 635s 64s/step - loss: 0.0085
Epoch 3/5
10/10 [==============================] - 635s 63s/step - loss: 0.0140
Epoch 4/5
10/10 [==============================] - 667s 67s/step - loss: 0.0098
Epoch 5/5
10/10 [==============================] - 985s 98s/step - loss: 0.0101


In [7]:
tobecolored = []
for filename in os.listdir('test/'):
    tobecolored.append(img_to_array(load_img('test/'+filename)))
tobecolored = np.array(tobecolored, dtype=float)
grayout = gray2rgb(rgb2gray(1 / 255 * tobecolored))
tobecolored_embed = inception_embeddings(grayout)
# 331*331*1...blah
tobecolored = rgb2lab(1 / 255 * tobecolored)[:,:,:,0]
tobecolored = tobecolored.reshape(tobecolored.shape+(1,))


# Test
output = model.predict([tobecolored, tobecolored_embed])
output = output * 128

# Output
for i in range(len(output)):
    done = np.zeros((256, 256, 3))
    done[:,:,0] = tobecolored[i][:,:,0]
    done[:,:,1:] = output[i]
    imsave("result/img_"+ str(i)+".png", lab2rgb(done))

D:\Anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
D:\Anaconda3\envs\tensorflow\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
